## MERGE

In [1]:
import numpy as np
import pandas as pd
import os

path = os.getcwd()

In [2]:
df_census = pd.read_csv(os.path.join(path, 'csv/census_by_city.csv'))     # (25235, 35)
df_crime = pd.read_csv(os.path.join(path, 'csv/fbi_crime_uscities.csv'))  # (8103, 16)
df_rent = pd.read_csv(os.path.join(path, 'csv/rent_cleaned.csv'))         # (2831, 4)
df_poll = pd.read_csv(os.path.join(path, 'csv/pollution_summary.csv'))    # (845, 20)
df_lat_lon = pd.read_excel(os.path.join(path, 'csv/uscities.xlsx'))       # (28338, 17)

In [3]:
# replace 'State_Id' in df_census so it matches the other columns
df_census.rename(columns = {'State_Id':'State'}, inplace = True)
df_lat_lon.rename(columns = {'state_id':'State','city': 'City'}, inplace = True)
df_lat_lon = df_lat_lon[['City', 'State', 'lat', 'lng']]

In [4]:
# put all the dfs in a list
df_lst = [df_census, df_crime, df_rent, df_poll, df_lat_lon]

# remove whitespace in all the dfs
for x in df_lst:
    x['City'] = x['City'].str.strip()
    x['State'] = x['State'].str.strip()

# make sure it worked    
for x in df_lst:
    print(x['City'][0])
    print(x['State'][0])

Anchor Point
AK
Anchorage
AK
Aleutians East
AK
Anchorage
AK
Adak
AK


In [5]:
# merging the data and following its shape sequentially
df_net = pd.merge(df_census, df_crime, on=['City', 'State'])
print(df_net.shape)
df_net2 = pd.merge(df_net, df_rent, on=['City', 'State'])
print(df_net2.shape)
df_net3 = pd.merge(df_net2, df_poll, on=['City', 'State'])
print(df_net3.shape)
df_net4 = pd.merge(df_net3, df_lat_lon, on=['City', 'State'])
print(df_net4.shape)

(6866, 49)
(768, 51)
(431, 69)
(425, 71)


In [6]:
df_net4 = df_net4.drop_duplicates()

## CLEANUP

### Update Crime Ratings 
- comparison is across fewer cities. 
- run pd.qcut again

In [7]:
df_net4['Crime Rating'].value_counts()

High      330
Medium     84
Low         8
Name: Crime Rating, dtype: int64

crime rating skews very high because after the merging there were only a little over 400 cities versus the 8000 cities reported in the FBI crime data. 

In [8]:
# dropping old crime rating column
df_net4 = df_net4.drop(columns = ['Crime Rating'])

In [9]:
# Create a new crime rating by using qcut() on the number of merged cities we have
df_net4['Crime Rating'] = pd.qcut(df_net4['Crime Rate (per 1000 residents)'], q=3, labels=['Low', 'Medium', 'High'])

In [10]:
df_net4['Crime Rating'].value_counts()

High      141
Low       141
Medium    140
Name: Crime Rating, dtype: int64

### Clean Columns

In [11]:
col = ["Poverty", "ChildPoverty", "Unemployment", "Professional", "Office", "Service", "Construction", "Production"]
df_net4[col] = df_net4[col].round(2)

### Add an Index Column

In [12]:
df_net4['Index'] = df_net4.index

### Reorder Columns

In [13]:
df_net4.columns

Index(['City', 'State', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
       'Black', 'Native', 'Asian', 'Pacific', 'Income', 'IncomeErr',
       'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty',
       'Employed', 'Unemployment', 'PrivateWork', 'PublicWork', 'SelfEmployed',
       'FamilyWork', 'Professional', 'Service', 'Office', 'Construction',
       'Production', 'Drive', 'Carpool', 'Transit', 'Walk', 'OtherTransp',
       'WorkAtHome', 'MeanCommute', 'Unnamed: 0_x', 'Population',
       'Violent crime', 'Murder and nonnegligent manslaughter', 'Rape',
       'Robbery', 'Aggravated assault', 'Property crime', 'Burglary',
       'Larceny- theft', 'Motor vehicle theft', 'Arson',
       'Crime Rate (per 1000 residents)', 'Unnamed: 0_y', 'Rent', 'Year',
       'Days with AQI', 'Good Days', 'Moderate Days',
       'Unhealthy for Sensitive Groups Days', 'Unhealthy Days',
       'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI', 'Median AQI', 

In [14]:
df_net4 = df_net4[[ 
    'Index',
    'City', 'State', 
    'lat', 'lng',
    'TotalPop', 'Men', 'Women', 'Hispanic', 'White',
    'Black', 'Native', 'Asian', 'Pacific', 'Income', 'IncomeErr','IncomePerCap', 
    'IncomePerCapErr', 'Poverty', 'ChildPoverty', 'Employed', 'Unemployment', 
    'PrivateWork', 'PublicWork', 'SelfEmployed', 'FamilyWork', 'Professional', 
    'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 
    'Transit', 'Walk', 'OtherTransp', 'WorkAtHome', 'MeanCommute', 
    'Rent', 'Year',
    'Population','Violent crime', 'Murder and nonnegligent manslaughter', 'Rape',
    'Robbery', 'Aggravated assault', 'Property crime', 'Burglary','Larceny- theft', 
    'Motor vehicle theft', 'Arson', 'Crime Rate (per 1000 residents)', 'Crime Rating',
    'Days with AQI', 'Good Days', 'Moderate Days', 'Unhealthy for Sensitive Groups Days', 
    'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI', 
    '90th Percentile AQI', 'Median AQI', 'Days CO', 'Days NO2', 'Days Ozone', 'Days SO2', 
    'Days PM2.5', 'Days PM10', 'Level of Concern',
    ]]

In [15]:
df_net4.rename(columns = {'Level of Concern':'Air Quality Index'}, inplace = True)

In [16]:
pd.set_option('display.max_columns', None)

In [17]:
df_net4.head()

,Index,City,State,lat,lng,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Employed,Unemployment,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Professional,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Rent,Year,Population,Violent crime,Murder and nonnegligent manslaughter,Rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft,Arson,Crime Rate (per 1000 residents),Crime Rating,Days with AQI,Good Days,Moderate Days,Unhealthy for Sensitive Groups Days,Unhealthy Days,Very Unhealthy Days,Hazardous Days,Max AQI,90th Percentile AQI,Median AQI,Days CO,Days NO2,Days Ozone,Days SO2,Days PM2.5,Days PM10,Air Quality Index
0,0,Anchorage,AK,61.1508,-149.1091,223042,113049,109977,8.0,55.6,5.4,8.0,9.6,2.8,57115.2,7787.0,26841.7,3621.2,586.0,790.2,118993,371.9,73.5,19.5,5.3,0.1,2434.4,1122.2,1508.9,461.0,571.5,74.3,10.9,2.2,3.3,3.7,3.8,17.1,1303,2020,287731,3581,32,540,621,2388,12261,1692,9038,1531,93,55.06,High,183,158,24,1,0,0,0,103,57,26,4,0,0,0,110,69,Good
1,1,Fairbanks,AK,64.8353,-147.6534,60751,32406,28341,5.5,71.2,3.2,6.9,2.8,0.6,41438.3,7688.9,20210.8,2646.6,170.9,198.2,30492,163.9,63.5,26.4,4.1,0.1,708.2,323.1,462.2,256.9,211.9,75.3,11.8,0.9,4.1,2.7,3.3,16.5,1380,2020,31493,247,3,33,52,159,1353,163,1006,184,23,50.80,High,183,132,41,3,7,0,0,167,89,35,1,0,81,1,97,3,Good
2,2,Juneau,AK,58.4546,-134.1739,30216,15465,14750,6.4,66.5,1.0,10.3,6.7,0.9,82937.5,10107.0,39731.2,4833.0,46.9,76.4,16669,27.0,52.3,38.7,8.9,0.1,282.1,88.0,122.8,63.2,43.8,64.3,14.6,5.0,9.1,2.5,4.5,15.3,1486,2020,31810,289,3,24,45,217,1292,187,1012,93,16,49.70,High,177,174,3,0,0,0,0,66,35,13,0,0,0,0,161,16,Good
3,3,Conway,AR,35.0754,-92.4694,82431,40485,41936,5.1,76.2,14.9,0.3,1.1,0.2,30933.4,5323.4,15551.0,2698.9,502.4,566.6,40133,150.3,79.0,14.9,5.5,0.1,903.9,512.9,666.6,277.2,314.3,80.7,10.2,0.1,2.8,2.1,3.4,22.0,898,2020,67336,324,2,49,51,222,1842,177,1567,98,8,32.17,Medium,275,207,66,2,0,0,0,132,59,42,1,4,138,0,132,0,Good
4,4,Conway,AR,35.0754,-92.4694,82431,40485,41936,5.1,76.2,14.9,0.3,1.1,0.2,30933.4,5323.4,15551.0,2698.9,502.4,566.6,40133,150.3,79.0,14.9,5.5,0.1,903.9,512.9,666.6,277.2,314.3,80.7,10.2,0.1,2.8,2.1,3.4,22.0,703,2020,67336,324,2,49,51,222,1842,177,1567,98,8,32.17,Medium,275,207,66,2,0,0,0,132,59,42,1,4,138,0,132,0,Good


In [18]:
# extract the merged df as a .csv
df_net4.to_csv('merged.csv', index = False) 